In [1]:
import tensorflow as tf

In [2]:
import numpy as np

In [3]:
tf.set_random_seed(777)

In [4]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
learning_rate = 0.001
training_epochs = 20
batch_size = 100

In [6]:
class Model:
    
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()
        
    def _build_net(self):
        with tf.variable_scope(self.name):
            self.training = tf.placeholder(tf.bool)
            self.X = tf.placeholder(tf.float32, [None, 784])
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])
            conv1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3, 3],
                                    padding="SAME", activation=tf.nn.relu)
            pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],
                                           padding="SAME", strides=2)
            dropout1 = tf.layers.dropout(inputs=pool1,
                                        rate=0.3, training=self.training)
            
            conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[3, 3],
                                    padding="SAME", activation=tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2],
                                           padding="SAME", strides=2)
            dropout2 = tf.layers.dropout(inputs=pool2, rate=0.3, training=self.training)
            
            conv3 = tf.layers.conv2d(inputs=dropout2, filters=128, kernel_size=[3, 3],
                                    padding="SAME", activation=tf.nn.relu)
            pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2],
                                           padding="SAME", strides=2)
            dropout3 = tf.layers.dropout(inputs=pool3, rate=0.3, training=self.training)
            
            flat = tf.reshape(dropout3, [-1, 128 * 4 * 4])
            
            dense4 = tf.layers.dense(inputs=flat, units=625, activation=tf.nn.relu)
            
            dropout4 = tf.layers.dropout(inputs=dense4, rate=0.5, training=self.training)
            
            self.logits = tf.layers.dense(inputs=dropout4, units=10)
            
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            logits=self.logits, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate=learning_rate).minimize(self.cost)
            
        correct_prediction = tf.equal(
            tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    def predict(self, x_test, training=False):
        return self.sess.run(self.logits,
                             feed_dict={self.X: x_test, self.training: training})
    
    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy,
                             feed_dict={self.X: x_test,
                                        self.Y: y_test, self.training: training})
    
    def train(self, x_data, y_data, training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict= {
            self.X: x_data, self.Y: y_data, self.training: training})

In [7]:
sess = tf.Session()
    
models = []
num_models = 2
    
for m in range(num_models):
    models.append(Model(sess, "model" + str(m)))
        
sess.run(tf.global_variables_initializer())

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [8]:
print("Learning started.")

for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c / total_batch
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', avg_cost_list)

print("Learning finished!")

test_size = len(mnist.test.labels)
predictions = np.zeros([test_size, 10])
for m_idx, m in enumerate(models):
    print(m_idx, "Accuracy:", m.get_accuracy(
        mnist.test.images, mnist.test.labels))
    p = m.predict(mnist.test.images)
    predictions += p
    
ensemble_correct_prediction = tf.equal(
    tf.argmax(predictions, 1), tf.argmax(mnist.test.labels, 1))
ensemble_accuracy = tf.reduce_mean(
    tf.cast(ensemble_correct_prediction, tf.float32))

print("Ensemble accuracy:", sess.run(ensemble_accuracy))

Learning started.
Epoch: 0001 cost = [0.28473004 0.29287701]
Epoch: 0002 cost = [0.08621121 0.08854457]
Epoch: 0003 cost = [0.06701532 0.06623238]
Epoch: 0004 cost = [0.05568747 0.05599411]
Epoch: 0005 cost = [0.04920513 0.05132549]
Epoch: 0006 cost = [0.04419062 0.04416698]
Epoch: 0007 cost = [0.0407241  0.04420776]
Epoch: 0008 cost = [0.0386727  0.03860363]
Epoch: 0009 cost = [0.03575879 0.03761801]
Epoch: 0010 cost = [0.03142363 0.03503302]
Epoch: 0011 cost = [0.03172802 0.03207466]
Epoch: 0012 cost = [0.03283399 0.03085349]
Epoch: 0013 cost = [0.02986531 0.02919762]
Epoch: 0014 cost = [0.02754508 0.02811268]
Epoch: 0015 cost = [0.02672134 0.02878029]
Epoch: 0016 cost = [0.02525713 0.02565511]
Epoch: 0017 cost = [0.02324561 0.02386417]
Epoch: 0018 cost = [0.02345328 0.02752146]
Epoch: 0019 cost = [0.02411527 0.0241042 ]
Epoch: 0020 cost = [0.02263397 0.02365217]
Learning finished!
0 Accuracy: 0.9941
1 Accuracy: 0.9936
Ensemble accuracy: 0.9944
